In [ ]:
import monaco as mc

# Import the statistical distributions from scipy.stats that you will be using.
# These must be rv_discrete or rv_continuous functions.
# See https://docs.scipy.org/doc/scipy/reference/stats.html for a complete list.
from scipy.stats import randint, rv_discrete

In [ ]:
# The preprocessing function should only take in an Case object, and extract the
# values from inside it in order to build the inputs for the run function.
def template_preprocess(case):

    # For all random variables that you initialized in your run script and will
    # be passed to your function, access them like so:
    flip = case.invals['flip'].val
    flipper = case.invals['flipper'].val

    # Import or declare any other unchanging inputs that your function needs as well.
    coin = 'quarter'

    # Structure your data, and return all the arguments to your sim function
    # packaged in a tuple. This tuple will be unpacked when your sim function
    # is called.
    return (flip, flipper, coin)

In [ ]:
# The run function input arguments need to match up with the outputs in the unpacked
# tuple from your preprocessing function
def template_run(flip, flipper, coin):

    # Your simulation calculations will happen here. This function can also be
    # a wrapper to call another function or non-python code.

    if flip == 0:
        headsortails = 'heads'
    elif flip == 1:
        headsortails = 'tails'

    simulation_output = {'headsortails': headsortails,
                         'flipper': flipper,
                         'coin': coin}

    # The outputs should be returned in a tuple, which will be unpacked when your
    # postprocessing function is called. Note the trailing comma to make this
    # tuple iterable.
    return (simulation_output, )

In [ ]:
# For your postprocessing function, the first argument must be the case, and
# all input arguments after case need to match up with the outputs in the unpacked
# tuple from your run function.
def template_postprocess(case, simulation_output):

    # Simulation outputs may be huge, and this is where postprocessing can be
    # done to extract the information you want. For example, you may only want
    # to know the last point in a timeseries.

    # It is good practice to provide a dictionary to map any non-number values
    # to numbers via a known valmap. If needed this will be auto-generated, but
    # manually assigning numbers will give greater control over plotting.
    valmap = {'heads': 0, 'tails': 1}

    # Add output values from this case's simulation results, case information,
    # or from other data processing that you do in this file.
    # The name supplied here will become the outvar's name.
    case.addOutVal(name='Flip Result', val=simulation_output['headsortails'], valmap=valmap)
    case.addOutVal(name='Flip Number', val=case.ncase)

In [ ]:
# These get packaged in the following format for Sim to consume:
fcns = {'preprocess' : template_preprocess,
        'run'        : template_run,
        'postprocess': template_postprocess}

# Set the number of random draws you wish to make.
# If firstcaseismedian is True, then case 0 will be run with the expected value of
# each statistical distribution as a 'nominal' run. The total number of cases
# will then be ndraws+1
ndraws = 500
firstcaseismedian = False

# Setting a known random seed is recommended for repeatability of random draws.
seed = 12362398

# Whether to run the simulation in a single thread. If True then the simulation
# will run single-threaded in a 'for' loop, and if False then parallel processing
# will be used. Parallel processing can be done either with dask or with python's
# in-built multiprocessing module, and this is controlled with the `usedask` flag.
# Note that depending on your simulation, the parallel processing may not be
# faster than the single-threaded version. The depends on the overhead of
# passing data between processes versus the speed of the simulation, and is
# worth experimenting with for your specific situation.
singlethreaded = True
usedask = False

# If you want, you can save all the results from each case to file, or just the
# postprocessed simulation results. This can be incredibly useful for examining
# time-consuming sim data after the fact without having to rerun the whole thing.
# On the other hand, if your run function generates a lot of data, you may not have
# enough storage space to save all of the raw case data. Plus, the file I/O may
# be the limiting factor that slows down your simulation. Try setting these flags
# to False and seeing how much faster the sim runs.
savecasedata = True
savesimdata = True

In [ ]:
# We first initialize the sim with a name of our choosing
sim = mc.Sim(name='Coin Flip', ndraws=ndraws, fcns=fcns,
             firstcaseismedian=firstcaseismedian, seed=seed,
             singlethreaded=singlethreaded,
             savecasedata=savecasedata, savesimdata=savesimdata,
             verbose=True, debug=False)

In [ ]:
# We now add input variables, with their associated distributions
# Out first variable will be the person flipping a coin - Sam and Alex will
# do so with even odds.
# For even odds between 0 and 1 we want to call scipy.stats.randint(0,2)
# The dist argument is therefor randint, and we look up its arguments in the
# documentation:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.randint.html
# The kwargs are {'low':0, 'high':2}, which we package in a keyword dictionary.
# Since our inputs here are strings rather than numbers, we need to map the
# numbers from our random draws to those imputs with a nummap dictionary
nummap = {0: 'Sam', 1: 'Alex'}
sim.addInVar(name='flipper', dist=randint, distkwargs={'low': 0, 'high': 2}, nummap=nummap)

# If we want to generate custom odds, we can create our own distribution, see:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_discrete.html
# Let's assume that we have a weighted coin where heads comes up 70% of the time.
# Our 'template_run' run function reads 0 as heads and 1 as tails.
flip_dist = rv_discrete(name='flip_dist', values=([0, 1], [0.7, 0.3]))
# If the distribution is custom then it doesn't take arguments, so pass in
# an empty dictionary
sim.addInVar(name='flip', dist=flip_dist, distkwargs=dict())

In [ ]:
# Once all input variables are initialized, we run the sim.
# For each case, the preprocessing function will pull in the random variables
# and any other data the run fuction needs, the run function executes, and
# its output is passed to the postprocessing function for extracting the
# desired output values from each case.
# The cases are collected, and the individual output values are collected into
# an output variable stored by the sim object.
sim.runSim()

In [ ]:
# Once the sim is run, we have access to its member variables.
print(f'{sim.name} Runtime: {sim.runtime}')

# From here we can perform further postprocessing on our results. The outvar
# names were assigned in our postprocessing function. We expect the heads
# bias to be near the 70% we assigned up in flip_dist.
bias = sim.outvars['Flip Result'].vals.count('heads')/sim.ncases*100
print(f'Average heads bias: {bias}%')

In [ ]:
# We can also quickly make some plots of our invars and outvars. The mc.plot
# function will automatically try to figure out which type of plot is most
# appropriate based on the number and dimension of the variables.
# This will make a histogram of the results:
mc.plot(sim.outvars['Flip Result'])

In [ ]:
# And this scatter plot will show that the flips were random over time:
mc.plot(sim.outvars['Flip Number'], sim.outvars['Flip Result'])

In [ ]:
# We can also look at the correlation between all scalar input and output
# vars to see which are most affecting the others. This shows that the input
# and output flip information is identical, and that the flipper and flip
# number had no effect on the coin landing heads or tails.
mc.plot_cov_corr(*sim.corr())